In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
import pandas as pd
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import linear_model, datasets
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
import matplotlib.pyplot as plt

In [ ]:
#main_df=pd.read_csv("creditcard.csv")
main_df=pd.read_csv("../input/creditcard.csv")
main_df.head()
feature_size = len(main_df.columns)
class_index = feature_size -1 

In [ ]:
fraud_raw = main_df[main_df['Class'] == 1]
normal_raw = main_df[main_df['Class'] == 0]

# Undersample the normal transactions
percentage = len(fraud_raw)/float(len(normal_raw))
print ('%0.4f'% percentage)

# the reason for testing different P/N class ratio dataset
After reviewing several notebooks, undersample strategy usually equal the numbers of two class dataset. Here I tried different class distribution to fit randomforrest models to investigate.  
The result shows that balanced(equal) data have the best recall score on test data.

In [ ]:
weights = [0.5,1,2,3,4,5,6,7,8,9,10,15,20]
for weight in weights:
    normal = normal_raw.sample(frac=weight*percentage)
    fraud = fraud_raw
    cc_data = fraud.append(normal)

    print ("Percentage of normal transactions                 :", len(normal) / float(len(cc_data)))
    print ("Percentage of fraudulent trasactions                :", len(fraud_raw) / float(len(cc_data)))
    print ("Total number of transactions in our new dataset :", len(cc_data))
    
    reg_model = RandomForestClassifier(criterion='entropy',n_estimators=100)
    train_data,test_data = train_test_split(cc_data,test_size=0.30)
    train_matrix = train_data.as_matrix()
    x = train_matrix[:,range(0,class_index-1)]
    y=train_matrix[:,class_index]
    reg_model = reg_model.fit(x,y)

    test_matrix =test_data.as_matrix()
    test_x = test_matrix[:,range(0,class_index-1)]
    test_y=test_matrix[:,class_index]
    
    predicted = reg_model.predict(test_x)
    
    print ('weight = ',weight)
    print ('recall = ',metrics.recall_score(test_y, predicted), 'precision = ',metrics.precision_score(test_y, predicted))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(test_y, predicted)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    print('AUC = %0.4f'% roc_auc)
    print ('------------------------')